In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
def write_flares(forf,cluster, test, EPIC, typ='flares'):
    
    '''
    Writes a data frame with flares or flux 
    for a certain light curve with EPIC ID from a cluster
    analysed during a certain test.
    
    Parameter:
    ------------
    
    Returns:
    ------------
    '''
    
    loc = '/home/ekaterina/Documents/appaloosa/stars_shortlist/{}/results/{}'.format(cluster,test)
    if typ == 'flares':
        forf.to_csv('{}/{}_flares.csv'.format(loc,EPIC))
    elif typ == 'flux':
        forf.to_csv('{}/{}_flux.csv'.format(loc,EPIC))
    return 

def find_nearest(array,value):
    '''
    
    Helper function that performs a binary search for the closest value in an array:
    
    Input:
    
    array - sorted array
    value - same type as elements in array
    
    Return:
    
    closest element in array
    index of the closest element in array
    
    '''

    idx = np.searchsorted(array, value, side="left")
    if idx > 0 and (idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx])):
        return array[idx-1], idx-1
    else:
        return array[idx], idx



def corr_fake(flares, EPIC, C, cluster, test):
    loc = '/home/ekaterina/Documents/appaloosa/stars_shortlist/'
    fakes = pd.read_csv('{}fakes/{}-c{}_kepler_v2.0_lc.fits_all_fakes.csv'.format(loc,EPIC,C),
               usecols=['ed_fake','rec_fake'])
    fakes = fakes.sort_values(by='ed_fake')[fakes.ed_fake < 10000]
    bins = np.power(10,np.arange(-2,5,0.15))
    fakes['range1'], bins = pd.cut(fakes.ed_fake, bins, retbins=True,include_lowest=True)

    m = pd.DataFrame()
    m['mean_ed_fake'] = fakes.groupby('range1').ed_fake.mean()
    m['mean_rec_fake'] = fakes.groupby('range1').rec_fake.mean()
    m['std_rec_fake'] = fakes.groupby('range1').rec_fake.std()
    m = m.dropna(how='any')

    m.plot('mean_ed_fake','mean_rec_fake',#yerr='std_rec_fake',
            xlim=(1,10000),ylim=(0,1.05),figsize=(6,3.5),label='recovery rate',logx=True,
            legend=False,title='EPIC 211423010')
    plt.xlabel('ED in s')
    plt.ylabel('recovery rate')
    #flares = pd.read_csv('{}{}/results/{}/{}_flares.csv'.format(loc,cluster,test,EPIC))
    plt.savefig('/home/ekaterina/Documents/Flares_in_Clusters_Paper/pic-dump/recover.jpg',dpi=300)
    corr = []
    for i in flares.ed:
        ed, id_ = find_nearest(np.asarray(m.mean_ed_fake),i) 
        corr.append(1./m.mean_rec_fake.iloc[id_])
    flares['corr'] = corr

    write_flares(flares,cluster,test,EPIC)
    return flares

In [2]:
loc = '/home/ekaterina/Documents/appaloosa/stars_shortlist/'
cluster = 'Pleiades'
C = '04'
test = 'run_01'
LCtype = 'everest'
EPIC = 211099781
flares = pd.read_csv('{}{}/results/{}/{}_flares.csv'.format(loc,cluster,test,EPIC))
flares[]
flares.head()

,Unnamed: 0,Unnamed: 0.1,ampl,dur,ed,ederr,istart,istop,corr
0,0,0,0.069520,0.490375,320.717917,9.295906,5,29,1.188571
1,1,1,0.001848,0.040863,3.560122,6.857952,3396,3398,62.200000


In [3]:
from fakescorr import fakescorr
fakescorr(flares, EPIC, C, cluster, test)

/home/ekaterina/Documents/appaloosa/appaloosa/fakescorr.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fakes = fakes.sort_values(by='ed_fake')[fakes.ed_fake < 20000]
/home/ekaterina/Documents/appaloosa/appaloosa/fakescorr.py:64: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fakes = fakes.sort_values(by='ed_fake')[fakes.ed_fake < 20000]
/home/ekaterina/Documents/appaloosa/appaloosa/fakescorr.py:71: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  fakes = fakes.sort_values(by='ed_fake')[fakes.ed_fake < 20000]


AttributeError: 'DataFrame' object has no attribute 'myed'